In [ ]:
import os
import datasets
from pathlib import Path

benchmark_dir = Path(os.environ['BENCHMARK_DIR'])

ds_key = 'flaviagiammarino/vqa-rad'
ds_name = ds_key.split('/')[-1]
ds = datasets.load_dataset(ds_key)

open_ended = list()
yes_no = list()

for split_name, split in ds.items():
  for item in split:
    item['split'] = split_name
    item['media'] = [item.pop('image')]
    
    if item['answer'].strip() in set(['yes', 'no']):
      item['answer'] = item['answer'].strip()[0].upper()
      yes_no.append(item)
    else:
      open_ended.append(item)

for idx, item in enumerate(yes_no):
    item['id'] = idx
for idx, item in enumerate(open_ended):
    item['id'] = idx

ds_yes_no = datasets.Dataset.from_list(yes_no)
ds_open_ended = datasets.Dataset.from_list(open_ended)

ds_yes_no.save_to_disk(benchmark_dir  / ds_name / 'yes-no' /'dataset')
ds_open_ended.save_to_disk(benchmark_dir / ds_name / 'open-ended'/'dataset')

Saving the dataset (0/1 shards):   0%|          | 0/1191 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1053 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "/model-weights/Qwen2.5-VL-7B-Instruct",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct", use_fast=True)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [15]:
del make_question

from infer import make_question

question, text = make_question(ds_yes_no[1], processor)
image_inputs, video_inputs = process_vision_info(question)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    return_tensors="pt"
).to(model.device)

generated_ids = model.generate(**inputs, max_new_tokens=32)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=False, clean_up_tokenization_spaces=False
)
print("Question:", question)
print("Input text:", text)
print(output_text)


Question: [{'role': 'user', 'content': [{'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x503 at 0x7FB0D41509D0>}, {'type': 'text', 'text': 'are the lungs normal appearing?\nYou only options are:\nY\nN\nPlease answer with exactly one letter chosen from the options above.'}]}]
Input text: <|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>are the lungs normal appearing?
You only options are:
Y
N
Please answer with exactly one letter chosen from the options above.<|im_end|>
<|im_start|>assistant

['N<|im_end|>']


In [16]:
ds_yes_no[1]

{'question': 'are the lungs normal appearing?',
 'answer': 'N',
 'split': 'train',
 'media': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x503>],
 'id': 1}

In [5]:
processor.batch_decode(inputs['input_ids'])

['<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|vision_end|>Describe this image.<|im_end|>\n

In [ ]:
in